In [3]:
import sqlite3
import pandas as pd
import os
from datetime import datetime

def connect_to_database(db_path):
    """
    Connect to the specified database
    """
    try:
        if not os.path.exists(db_path):
            print(f"❌ Database file not found: {db_path}")
            return None
        
        conn = sqlite3.connect(db_path)
        print(f"✅ Successfully connected to database: {db_path}")
        return conn
    except sqlite3.Error as e:
        print(f"❌ Error connecting to database: {e}")
        return None

def analyze_database_structure(conn):
    """
    Analyze the database to find tables with date and buy columns
    """
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        print(f"📊 Found {len(tables)} table(s) in the database")
        
        suitable_tables = {}
        
        for table in tables:
            table_name = table[0]
            print(f"\n🔸 Analyzing table: {table_name}")
            
            # Get column info
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            column_names = [col[1].lower() for col in columns]
            original_columns = [col[1] for col in columns]
            
            # Look for date columns
            date_columns = [col for col in original_columns if any(keyword in col.lower() 
                           for keyword in ['date', 'time', 'timestamp', 'day'])]
            
            # Look for buy columns
            buy_columns = [col for col in original_columns if any(keyword in col.lower() 
                          for keyword in ['buy', 'purchase', 'bid'])]
            
            # Look for instrument columns
            instrument_columns = [col for col in original_columns if any(keyword in col.lower() 
                                 for keyword in ['symbol', 'instrument', 'ticker', 'asset', 'code'])]
            
            print(f"   Date columns found: {date_columns}")
            print(f"   Buy columns found: {buy_columns}")
            print(f"   Instrument columns found: {instrument_columns}")
            
            if date_columns and buy_columns and instrument_columns:
                # Get row count
                cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
                row_count = cursor.fetchone()[0]
                
                suitable_tables[table_name] = {
                    'date_columns': date_columns,
                    'buy_columns': buy_columns,
                    'instrument_columns': instrument_columns,
                    'row_count': row_count,
                    'all_columns': original_columns
                }
                print(f"   ✅ Suitable table with {row_count:,} rows")
            else:
                print(f"   ⚠️  Missing required columns")
        
        return suitable_tables
        
    except sqlite3.Error as e:
        print(f"❌ Error analyzing database: {e}")
        return None

def analyze_instruments_date_range(conn, suitable_tables):
    """
    Analyze date ranges and row counts for each instrument in suitable tables
    """
    try:
        cursor = conn.cursor()
        
        print(suitable_tables.items())
        for table_name, table_info in suitable_tables.items():
            print(f"\n📈 Analyzing instruments in table: {table_name}")
            print("-" * 50)
            
            # Get the first date column and first instrument column
            date_col = table_info['date_columns'][0]
            instrument_col = table_info['instrument_columns'][0]
            
            # Query to get date range and count for each instrument
            query = f"""
            SELECT 
                {instrument_col} as instrument,
                MIN({date_col}) as start_date,
                MAX({date_col}) as end_date,
                COUNT(*) as row_count
            FROM {table_name}
            WHERE {instrument_col} IS NOT NULL
            GROUP BY {instrument_col}
            ORDER BY {instrument_col}
            """

            print(f"Executing query:\n{query}\n")
            
            cursor.execute(query)
            results = cursor.fetchall()

            print(results)
            
            if results:
                print(f"{'Instrument':<15} {'Start Date':<12} {'End Date':<12} {'Rows':<10}")
                print("-" * 55)
                
                for row in results:
                    instrument, start_date, end_date, count = row
                    print(f"{instrument:<15} {start_date:<12} {end_date:<12} {count:<10,}")
                
                print(f"\n📊 Total unique instruments: {len(results)}")
                total_rows = sum(row[3] for row in results)
                print(f"📊 Total rows across all instruments: {total_rows:,}")
            else:
                print("⚠️  No instruments found in this table")
        
    except sqlite3.Error as e:
        print(f"❌ Error analyzing instruments: {e}")

# Main execution
db_path = "weekly/db/aia_big.db"

# Connect to database
conn = connect_to_database(db_path)

if conn:
    print("\n" + "="*60)
    print("🔍 ANALYZING DATABASE STRUCTURE")
    print("="*60)
    
    # Analyze database structure
    suitable_tables = analyze_database_structure(conn)
    
    if suitable_tables:
        print("\n" + "="*60)
        print("📅 ANALYZING INSTRUMENT DATE RANGES")
        print("="*60)
        
        # Analyze date ranges for each instrument
        analyze_instruments_date_range(conn, suitable_tables)
    
    conn.close()
    print(f"\n✅ Database connection closed")
else:
    print("❌ Could not connect to database")

✅ Successfully connected to database: weekly/db/aia_big.db

🔍 ANALYZING DATABASE STRUCTURE
📊 Found 1 table(s) in the database

🔸 Analyzing table: gather
   Date columns found: ['date']
   Buy columns found: ['buy']
   Instrument columns found: ['instrument']
   ✅ Suitable table with 36,862,289 rows

📅 ANALYZING INSTRUMENT DATE RANGES
dict_items([('gather', {'date_columns': ['date'], 'buy_columns': ['buy'], 'instrument_columns': ['instrument'], 'row_count': 36862289, 'all_columns': ['date', 'instrument', 'buy', 'sell']})])

📈 Analyzing instruments in table: gather
--------------------------------------------------
Executing query:

            SELECT 
                instrument as instrument,
                MIN(date) as start_date,
                MAX(date) as end_date,
                COUNT(*) as row_count
            FROM gather
            WHERE instrument IS NOT NULL
            GROUP BY instrument
            ORDER BY instrument
            

[('AU200_AUD', '2025-06-27 20:59:05.30